In [1]:
import pandas as pd
dados = pd.read_csv("Manaus.csv", encoding ='ISO-8859-1', sep = ";", low_memory = False )
#dados.columns
# low_memory = False só para tirar o aviso gerado ao ler diferentes dtypes em uma coluna de um arquivo.
# mas talvez seja necessário especificar os tipos de dados de "_srag" e "_evolucao".

In [2]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108351 entries, 0 to 108350
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _idade                108230 non-null  float64
 1   _faixa etária         108351 non-null  object 
 2   _sexo                 107551 non-null  object 
 3   _bairro               106223 non-null  object 
 4   _classificacao        108351 non-null  object 
 5   _comorb_renal         94992 non-null   object 
 6   _comorb_diabetes      95646 non-null   object 
 7   _comorb_imuno         94859 non-null   object 
 8   _comorb_cardio        95698 non-null   object 
 9   _conclusao            64812 non-null   object 
 10  _dt_notificacao       108340 non-null  object 
 11  _taxa                 102551 non-null  object 
 12  _dt_evolucao          39685 non-null   object 
 13  _raca                 83920 non-null   object 
 14  _dt_sintomas          104710 non-null  object 
 15  

In [3]:
dados.head()

,_idade,_faixa etária,_sexo,_bairro,_classificacao,_comorb_renal,_comorb_diabetes,_comorb_imuno,_comorb_cardio,_conclusao,...,_distrito,_bairro_mapa,_comorb_respiratoria,_comorb_cromossomica,_comorb_hepatica,_comorb_neurologica,_comorb_hemato,_comorb_obessidade,_origem,_evolução
0,37.0,30a - 39a,F,CONJ. CIDADAO 10,Confirmado,Não,Não,Não,Não,NaN,...,OESTE,TARUMÃ,Não,Não,NaN,NaN,NaN,NaN,"eSUS VE,",NaN
1,46.0,40a - 49a,F,MORADA DO SOL,Descartado,Não,Não,Não,Não,Recuperado,...,SUL,ALEIXO,Não,Não,NaN,NaN,NaN,NaN,"eSUS VE,",NaN
2,51.0,50a - 59a,F,NaN,Confirmado,Não,Não,Não,Não,NaN,...,NaN,NaN,Não,Não,NaN,NaN,NaN,NaN,"eSUS VE,",NaN
3,74.0,70a - 79a,M,NaN,Confirmado,Não,Não,Não,Não,NaN,...,NaN,NaN,Não,Não,NaN,NaN,NaN,NaN,"eSUS VE,",NaN
4,51.0,50a - 59a,F,ALEIXO,Confirmado,Não,Não,Não,Não,NaN,...,SUL,ALEIXO,Não,Não,NaN,NaN,NaN,NaN,"eSUS VE,",NaN


In [4]:
len(dados) 

108351

Considerar apenas os casos confirmados, excluindo todos os demais registros distintos.

In [5]:
dados._classificacao.unique()

array(['Confirmado', 'Descartado', 'Em análise'], dtype=object)

In [6]:
#selecionando apenas os casos confirmados
casos_confirmados = dados.query("_classificacao == 'Confirmado'")

**Quantos atributos descrevem cada exemplo? Quais são eles?**
36 atributos. 

In [7]:
casos_confirmados.columns

Index(['_idade', '_faixa etária', '_sexo', '_bairro', '_classificacao',
       '_comorb_renal', '_comorb_diabetes', '_comorb_imuno', '_comorb_cardio',
       '_conclusao', '_dt_notificacao', '_taxa', '_dt_evolucao', '_raca',
       '_dt_sintomas', '_criterio', '_tipo_teste', '_sintoma_garganta',
       '_sintoma_dispneia', '_sintoma_febre', '_sintoma_tosse',
       '_sintoma_outros', '_etnia', '_profiss_saude', '_srag',
       '_se_notificacao', '_distrito', '_bairro_mapa', '_comorb_respiratoria',
       '_comorb_cromossomica', '_comorb_hepatica', '_comorb_neurologica',
       '_comorb_hemato', '_comorb_obessidade', '_origem', '_evolução'],
      dtype='object')

**Quantos casos confirmados há em Manaus, cumulativamente?**

In [8]:
len(casos_confirmados)

36947

**A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o
mais recente? Leve em conta a data de notificação.**

Desde 1 de abril até 12 de julho de 2020.

In [9]:
#precisa tratar data
cc = pd.DataFrame(casos_confirmados)

cc["_dt_notificacao"] = pd.to_datetime(cc["_dt_notificacao"],)

cc = cc.sort_values(by="_dt_notificacao")

cc["_dt_notificacao"].dropna() # Tinham dados sem a data

102714   2020-01-04
89228    2020-01-04
1363     2020-01-04
6980     2020-01-04
68771    2020-01-04
            ...    
54360    2020-12-07
88748    2020-12-07
49129    2020-12-07
52399    2020-12-07
78933    2020-12-07
Name: _dt_notificacao, Length: 36943, dtype: datetime64[ns]

**Vamos excluir todos os atributos relativos às comorbidades, sintomas, etnia, profissão, outras datas que não a de notificação, origem e outros que não estiverem envolvidos no contexto do trabalho solicitado. Estes atributos serão considerados irrelevantes para fins de simplificação. Exclua todas as linhas em que houver dados faltantes para os atributos remanescentes.**

In [10]:
dados_copia = casos_confirmados.drop(columns=['_comorb_renal', '_comorb_diabetes', '_comorb_imuno',
                                              '_comorb_cardio', '_comorb_cromossomica', '_comorb_hepatica',
                                              '_comorb_neurologica','_comorb_hemato', '_comorb_obessidade',
                                              '_etnia', '_sintoma_garganta', '_sintoma_dispneia', '_sintoma_febre',
                                              '_sintoma_tosse', '_sintoma_outros', '_dt_evolucao', '_dt_sintomas',
                                              '_profiss_saude', '_origem', '_comorb_respiratoria', '_taxa',
                                              '_criterio','_raca'])
dados_copia.columns

Index(['_idade', '_faixa etária', '_sexo', '_bairro', '_classificacao',
       '_conclusao', '_dt_notificacao', '_tipo_teste', '_srag',
       '_se_notificacao', '_distrito', '_bairro_mapa', '_evolução'],
      dtype='object')

In [11]:
#analisar se precisa dos atributos "_se_notificacao" "evolucao"

In [12]:
dados_copia.isnull().sum()

_idade                57
_faixa etária          0
_sexo                162
_bairro              582
_classificacao         0
_conclusao         23567
_dt_notificacao        4
_tipo_teste         8103
_srag              31696
_se_notificacao        4
_distrito           1795
_bairro_mapa        1795
_evolução          31501
dtype: int64

In [13]:
dados_copia.head()

,_idade,_faixa etária,_sexo,_bairro,_classificacao,_conclusao,_dt_notificacao,_tipo_teste,_srag,_se_notificacao,_distrito,_bairro_mapa,_evolução
0,37.0,30a - 39a,F,CONJ. CIDADAO 10,Confirmado,NaN,2020-04-05,TESTE RÁPIDO - ANTÍGENO,NaN,19.0,OESTE,TARUMÃ,NaN
2,51.0,50a - 59a,F,NaN,Confirmado,NaN,2020-04-05,TESTE RÁPIDO - ANTICORPO,NaN,19.0,NaN,NaN,NaN
3,74.0,70a - 79a,M,NaN,Confirmado,NaN,2020-04-05,TESTE RÁPIDO - ANTICORPO,NaN,19.0,NaN,NaN,NaN
4,51.0,50a - 59a,F,ALEIXO,Confirmado,NaN,2020-04-05,TESTE RÁPIDO - ANTICORPO,NaN,19.0,SUL,ALEIXO,NaN
5,53.0,50a - 59a,M,PARQUE10,Confirmado,NaN,2020-04-05,TESTE RÁPIDO - ANTICORPO,NaN,19.0,SUL,PARQUE 10 DE NOVEMBRO,NaN


In [14]:
#excluindo dados nulos
#dados_copia.dropna()

**1. Quantos exemplos e atributos há na base de dados após a limpeza e organização?**

In [17]:
dados_copia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36947 entries, 0 to 108349
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   _idade           36890 non-null  float64       
 1   _faixa etária    36947 non-null  object        
 2   _sexo            36785 non-null  object        
 3   _bairro          36365 non-null  object        
 4   _classificacao   36947 non-null  object        
 5   _conclusao       13380 non-null  object        
 6   _dt_notificacao  36943 non-null  datetime64[ns]
 7   _tipo_teste      28844 non-null  object        
 8   _srag            5251 non-null   object        
 9   _se_notificacao  36943 non-null  float64       
 10  _distrito        35152 non-null  object        
 11  _bairro_mapa     35152 non-null  object        
 12  _evolução        5446 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(10)
memory usage: 3.9+ MB


**2. Qual a porcentagem de indivíduos recuperados em relação ao todo?**

In [21]:
casos_recuperados = dados_copia.query("_conclusao == 'Recuperado'")

len(casos_recuperados)

11340

**3. Os casos acometeram mais indivíduos do sexo masculino ou feminino?**

In [23]:
cc_M = dados_copia.query("_sexo == 'M'")
cc_F = dados_copia.query("_sexo == 'F'")

print("Masculino:",len(cc_M))
print("Feminino:",len(cc_F))

Masculino: 17410
Feminino: 19374


Indíviduos do sexo feminino.

**4. Qual a média e desvio padrão de idade dos indivíduos que contraíram COVID-19? Qual o
indivíduo mais jovem e o mais idoso a contraírem tal enfermidade?**

**5. Qual o bairro com maior incidência de casos?**

**6. Quais os três bairros com maior incidência de casos recuperados?**


**7. Quais os tipos de testes efetuados, segundo os dados? Indique os dados de maneira quantitativa
e percentual.**


**8. Qual taxa de letalidade pode ser calculada a partir do conjunto de dados? Para calcular esta
taxa, considere a fração do total de óbitos pelo total de casos;**


**9. Qual o tipo de correlação, mediante coeficiente de correlação de Pearson, entre a idade e o
número de casos? Para responder a esta pergunte, agrupe o número de casos por idade e efetue
o cálculo de tal coeficiente. Indique, a partir do resultado, a natureza desta correlação, se é
positiva ou negativa, e qual sua intensidade.**